### Main module that demonstrate the current compiler

The current project contains a runtime library (please see the runtime folder), which define several functions for IO and string manipulation, and a compiler which compile COOL program to x64(intel) assembly.

## I have only tested my compiler on MAC OX X with python 3.

To use this compiler please make sure you do the following steps:

1. navigate to the folder(the root folder) which contains all the python file.
2. then run the command `python3 main.py <filename>.cl`, this will generate the assembly file `<filename>.s` in the root folder
3. run 
    `clang(or gcc) runtime.c startup.s <filename>.s -o <filename>` 
to generate binary file
3. run `./<filename>` to see the result.


#### The current compiler did not implement the following features due to time constaint.

1. case expression
2. null pointer dereference detection and handling.
3. no garbage collector
4. no optimization

I also did not get the chance to properly test my compiler so I will demonstate this compiler with a few examples.
  

In [15]:
from parser import *
import cool_global as GLOBAL
from cool_codegen import *

def compile(code):
    parser = make_parser()
    
    parse_result = parser.parse(code)
    type_scope = parse_result.typecheck()
    cgen = CGen(parse_result, type_scope)
    code = cgen.code_gen()
    
    print(code)

    

In [16]:
compile("""
class Main inherits IO {
   
   main(): SELF_TYPE {
	out_string("Hello, World.\n \n\n") -- the "" is needed to satisfy the typchecker
   };
};


""")

.data

    .globl Main_init
    .globl Main_protoObj
    .globl Main_main


class_objTab:
    .quad    Object_protoObj
    .quad    Object_init
    .quad    IO_protoObj
    .quad    IO_init
    .quad    String_protoObj
    .quad    String_init
    .quad    Int_protoObj
    .quad    Int_init
    .quad    Bool_protoObj
    .quad    Bool_init
    .quad    Main_protoObj
    .quad    Main_init

Object_protoObj:
    .quad    0
    .quad    3
    .quad    Object_dispatch_table

IO_protoObj:
    .quad    1
    .quad    3
    .quad    IO_dispatch_table

String_protoObj:
    .quad    2
    .quad    5
    .quad    String_dispatch_table
    .quad    0
    .quad    0

Int_protoObj:
    .quad    3
    .quad    4
    .quad    Int_dispatch_table
    .quad    0

Bool_protoObj:
    .quad    4
    .quad    4
    .quad    Bool_dispatch_table
    .quad    0

Main_protoObj:
    .quad    5
    .quad    3
    .quad    Main_dispatch_table

IO_dispatch_table:
    .quad    Object_abort
    .quad    Object_copy
 

Generating LALR tables


### clang runtime/runtime.c runtime/startup.s x86/hellow_world.s -o bin/hello_world

Hello, World.



COOL program successfully executed

# If statement

In [17]:
compile("""
class Main inherits IO {
   
   main(): SELF_TYPE {
       if false
       then out_string("this is true\n")
       else out_string("this is false\n")
       fi
   };
};


""")

.data

    .globl Main_init
    .globl Main_protoObj
    .globl Main_main


class_objTab:
    .quad    Object_protoObj
    .quad    Object_init
    .quad    IO_protoObj
    .quad    IO_init
    .quad    String_protoObj
    .quad    String_init
    .quad    Int_protoObj
    .quad    Int_init
    .quad    Bool_protoObj
    .quad    Bool_init
    .quad    Main_protoObj
    .quad    Main_init

Object_protoObj:
    .quad    0
    .quad    3
    .quad    Object_dispatch_table

IO_protoObj:
    .quad    1
    .quad    3
    .quad    IO_dispatch_table

String_protoObj:
    .quad    2
    .quad    5
    .quad    String_dispatch_table
    .quad    0
    .quad    0

Int_protoObj:
    .quad    3
    .quad    4
    .quad    Int_dispatch_table
    .quad    0

Bool_protoObj:
    .quad    4
    .quad    4
    .quad    Bool_dispatch_table
    .quad    0

Main_protoObj:
    .quad    5
    .quad    3
    .quad    Main_dispatch_table

IO_dispatch_table:
    .quad    Object_abort
    .quad    Object_copy
 

Generating LALR tables


## clang runtime/runtime.c runtime/startup.s x86/if.s -o bin/if

this is false

COOL program successfully executed

# While loop

In [18]:
compile("""
class Main inherits IO{
	x : Int;
	main () : Object {
		{
			while x<10 loop 
			{
				out_int(x) ;
				x <- x + 1;
			}
			pool;
		}
	};
};


""")

Generating LALR tables


.data

    .globl Main_init
    .globl Main_protoObj
    .globl Main_main


class_objTab:
    .quad    Object_protoObj
    .quad    Object_init
    .quad    IO_protoObj
    .quad    IO_init
    .quad    String_protoObj
    .quad    String_init
    .quad    Int_protoObj
    .quad    Int_init
    .quad    Bool_protoObj
    .quad    Bool_init
    .quad    Main_protoObj
    .quad    Main_init

Object_protoObj:
    .quad    0
    .quad    3
    .quad    Object_dispatch_table

IO_protoObj:
    .quad    1
    .quad    3
    .quad    IO_dispatch_table

String_protoObj:
    .quad    2
    .quad    5
    .quad    String_dispatch_table
    .quad    0
    .quad    0

Int_protoObj:
    .quad    3
    .quad    4
    .quad    Int_dispatch_table
    .quad    0

Bool_protoObj:
    .quad    4
    .quad    4
    .quad    Bool_dispatch_table
    .quad    0

Main_protoObj:
    .quad    5
    .quad    4
    .quad    Main_dispatch_table
    .quad    0

IO_dispatch_table:
    .quad    Object_abort
    .quad  

## clang runtime/runtime.c runtime/startup.s x86/while.s -o bin/while

0

1

2

3

4

5

6

7

8

9

COOL program successfully executed

# Dynamic dispatch

In [19]:
compile ("""
class A inherits IO {

    init() : Object {
        out_string("init A")
    };

    toString() : Object {
        out_string("toString A called")
    };


};

class B inherits A {

    init() : Object  {
        out_string("init B")
    };

    toString() : Object {
        out_string("toString B called")
    };
    

};


class Main inherits IO {

   a : A;
   
   main(): Object {
       {
        a <- new B;
        a.toString();
       }
   };
};


""")

.data

    .globl Main_init
    .globl Main_protoObj
    .globl Main_main


class_objTab:
    .quad    Object_protoObj
    .quad    Object_init
    .quad    IO_protoObj
    .quad    IO_init
    .quad    String_protoObj
    .quad    String_init
    .quad    Int_protoObj
    .quad    Int_init
    .quad    Bool_protoObj
    .quad    Bool_init
    .quad    A_protoObj
    .quad    A_init
    .quad    B_protoObj
    .quad    B_init
    .quad    Main_protoObj
    .quad    Main_init

Object_protoObj:
    .quad    0
    .quad    3
    .quad    Object_dispatch_table

IO_protoObj:
    .quad    1
    .quad    3
    .quad    IO_dispatch_table

String_protoObj:
    .quad    2
    .quad    5
    .quad    String_dispatch_table
    .quad    0
    .quad    0

Int_protoObj:
    .quad    3
    .quad    4
    .quad    Int_dispatch_table
    .quad    0

Bool_protoObj:
    .quad    4
    .quad    4
    .quad    Bool_dispatch_table
    .quad    0

A_protoObj:
    .quad    5
    .quad    3
    .quad    A_dispa

Generating LALR tables


## clang runtime/runtime.c runtime/startup.s x86/dynamic.s -o bin/dynamic

init A

init B

toString B called

COOL program successfully executed

# Let expression - lexical scope

In [20]:
compile("""
class Main inherits IO {
    main() : Object {
        let x : Int <- 3 in
        {
            out_int(x);
            {let x : Int <- 1 in
                out_int(x);
            };
            out_int(x);

        }
    };
};
""")

.data

    .globl Main_init
    .globl Main_protoObj
    .globl Main_main


class_objTab:
    .quad    Object_protoObj
    .quad    Object_init
    .quad    IO_protoObj
    .quad    IO_init
    .quad    String_protoObj
    .quad    String_init
    .quad    Int_protoObj
    .quad    Int_init
    .quad    Bool_protoObj
    .quad    Bool_init
    .quad    Main_protoObj
    .quad    Main_init

Object_protoObj:
    .quad    0
    .quad    3
    .quad    Object_dispatch_table

IO_protoObj:
    .quad    1
    .quad    3
    .quad    IO_dispatch_table

String_protoObj:
    .quad    2
    .quad    5
    .quad    String_dispatch_table
    .quad    0
    .quad    0

Int_protoObj:
    .quad    3
    .quad    4
    .quad    Int_dispatch_table
    .quad    0

Bool_protoObj:
    .quad    4
    .quad    4
    .quad    Bool_dispatch_table
    .quad    0

Main_protoObj:
    .quad    5
    .quad    3
    .quad    Main_dispatch_table

IO_dispatch_table:
    .quad    Object_abort
    .quad    Object_copy
 

Generating LALR tables


## clang runtime/runtime.c runtime/startup.s x86/let.s -o bin/let

3

1

3
COOL program successfully executed